In [1]:
%matplotlib inline
#import data 

import pathlib
import imageio
import numpy as np

# Glob the training data and load a single image path
training_paths = pathlib.Path('/modules/cs342/Assignment2/FullTraining').glob('*/images/*.png')
training_sorted = sorted([x for x in training_paths])

masks_paths = pathlib.Path('/modules/cs342/Assignment2/FullTraining').glob('*/masks/*.png')
masks_sorted = sorted([x for x in masks_paths])

im_path = training_sorted[45]
im = imageio.imread(str(im_path))

print len(masks_sorted)
im_path2 = masks_sorted[45]
im2 = imageio.imread(str(im_path2))

IndexError: list index out of range

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(im)

In [ ]:
from skimage.color import rgb2gray
im_gray = rgb2gray(im)
plt.imshow(im_gray, cmap='gray')

<large>HoG</large>

In [ ]:
from skimage.feature import hog
from skimage import data, exposure

fd,hog_image = hog(im_gray, orientations=16, pixels_per_cell=(2, 2),cells_per_block=(3, 3),visualise=True)

In [ ]:
plt.imshow(hog_image,cmap='gray', interpolation='nearest');

winSize = (32,32)
blockSize = (32,32)
blockStride = (2,2)
cellSize = (2,2)
nbins = 9

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)
hist = hog.compute(im
                  )


It looks like hog is not very well suited for this kind of problem. Anyway just say in the report not very useful for our problem.

<large>Otsu threshold + watershed</large>

In [ ]:
#code taken from kernel

from skimage.filters import threshold_otsu
thresh_val = threshold_otsu(im_gray)
mask = np.where(im_gray > thresh_val, 1, 0)

# Make sure the larger portion of the mask is considered background
if np.sum(mask==0) < np.sum(mask==1):
    mask = np.where(mask, 0, 1)



In [ ]:
plt.imshow(mask)

In [ ]:
from scipy import ndimage
labels, nlabels = ndimage.label(mask)

label_arrays = []
for label_num in range(1, nlabels+1):
    label_mask = np.where(labels == label_num, 1, 0)
    label_arrays.append(label_mask)

print('There are {} separate components / objects detected.'.format(nlabels))




In [ ]:
plt.imshow(label_arrays[30])

In [ ]:
plt.imshow(labels)

In [ ]:
# Create a random colormap
from matplotlib.colors import ListedColormap
rand_cmap = ListedColormap(np.random.rand(256,3))

labels_for_display = np.where(labels > 0, labels, np.nan)
plt.imshow(im_gray, cmap='gray')
plt.imshow(labels_for_display, cmap=rand_cmap)
plt.axis('off')
plt.title('Labeled Cells ({} Nuclei)'.format(nlabels))
plt.show()

In [ ]:
for label_ind, label_coords in enumerate(ndimage.find_objects(labels)):
    cell = im_gray[label_coords]
    
    # Check if the label size is too small
    if np.product(cell.shape) < 10: 
        print('Label {} is too small! Setting to 0.'.format(label_ind))
        mask = np.where(labels==label_ind+1, 0, mask)

# Regenerate the labels
labels, nlabels = ndimage.label(mask)
print('There are now {} separate components / objects detected.'.format(nlabels))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max

image=labels

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray, interpolation='nearest')
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest')
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()


From the above we can see that we can also do watershed but it doesn't look very good, a lot of nuclei are further split.

In [ ]:
import pandas as pd
import os
train_labels = pd.read_csv(os.path.join('/modules/cs342/Assignment2/FullTraining','{}_train_labels.csv'.format('stage1')))
train_labels['EncodedPixels'] = train_labels['EncodedPixels'].map(lambda ep: [int(x) for x in ep.split(' ')])
train_labels.head(27)



In [ ]:
train_labels['EncodedPixels'][2]

We need to rewrite all the part above, but for the moment not important.
Now we focus on the second part. Try to use MLP. But to do this we first need to scale all pictures to the same.
We try to using code from: https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

In [ ]:
# Set some parameters
from skimage.transform import resize
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 4
TRAIN_PATH = '/modules/cs342/Assignment2/FullTraining'
TEST_PATH = '../input/stage1_test/'


scaled_pictures = np.zeros((len(training_sorted), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
for n,im_path in enumerate(training_sorted):
    im = imageio.imread(str(im_path))
    im = resize(im, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    scaled_pictures[n]= im
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in masks_sorted:
        mask
        
    
    
scaled_masks = np.zeros((len(test_sorted), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

for n,im_path in enumerate(training_sorted):
    im = imageio.imread(str(im_path))
    im = resize(im, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    scaled_pictures[n]= im

    
    

In [ ]:
import os
import sys
import random
import warnings

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '/modules/cs342/Assignment2/FullTraining/'
TEST_PATH = '/modules/cs342/Assignment2/FullTesting/'

train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]


# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img


In [ ]:
plt.imshow(Y_train[10][:,:,0])

In [ ]:
plt.imshow(X_train[10])

In [ ]:
(Y_train[23]).shape

In [ ]:
Y_train[0]

In [ ]:
pixels = Y_train[10].flatten()

In [ ]:
pixels

In [ ]:
X_train_pixels= []
for picture in X_train:
    X_train_pixels.append(picture.flatten())
X_train_pixels= np.asarray(X_train_pixels)    
    
   
Y_train_pixels= []
for picture in Y_train:
    Y_train_pixels.append(picture.flatten())
Y_train_pixels= np.asarray(Y_train_pixels)  
    
X_test_pixels= []
for picture in X_test:
    X_test_pixels.append(picture.flatten()) 
X_test_pixels= np.asarray(X_test_pixels)  

In [ ]:
len(X_test_pixels)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_train_pixels,Y_train_pixels)


In [ ]:
len(Y_train_pixels[2])

In [ ]:
a = mlp.predict(X_train_pixels[6])[0]

In [ ]:
a=a.reshape((128,128))

In [ ]:
plt.imshow(a)

In [ ]:
b =X_test_pixels[8]

In [ ]:
b.reshape((128,128))

In [ ]:
len(b)